In [1]:
# 安装PySpark（如果Colab环境未预装）
!pip install pyspark findspark -q

# 导入库并初始化Spark
from pyspark.sql import SparkSession
import findspark
findspark.init()

# 创建SparkSession，这是所有Spark功能的入口
# 我们启用Hive支持，以便后续模拟Hive操作
spark = SparkSession.builder \
    .appName("AmazonReviewAnalysis") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# 检查Spark是否正常运行
print(f"Spark版本: {spark.version}")
print("SparkSession创建成功！")
sc = spark.sparkContext
print(f"SparkContext运行模式: {sc.master}")

Spark版本: 4.0.1
SparkSession创建成功！
SparkContext运行模式: local[*]


In [4]:
!wget -q http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
print("下载完成，正在解压...")
!gzip -d reviews_Electronics_5.json.gz
print("解压完成！")

# 现在加载这个文件
file_path = 'reviews_Electronics_5.json'
raw_df = spark.read.json(file_path)

下载完成，正在解压...
解压完成！


In [5]:
raw_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [6]:
# 1. 检查数据规模
total_count = raw_df.count()
print(f"✅ 数据总行数: {total_count:,}")

# 2. 检查核心字段的缺失值（直接影响任务）
print("\n🔍 核心字段空值检查:")
critical_cols = ["reviewText", "overall", "asin", "reviewerID"]
for col in critical_cols:
    null_count = raw_df.filter(raw_df[col].isNull()).count()
    print(f"   {col}: 空值数 {null_count:,} ({null_count/total_count*100:.2f}%)")

# 3. 检查评分范围是否合法（应为1-5）
print("\n📊 评分(overall)分布检查:")
raw_df.groupBy("overall").count().orderBy("overall").show()

# 4. 解析helpful数组示例（后续计算的关键）
print("\n📈 helpful字段示例（[有用票数， 总票数]）:")
raw_df.select("helpful").limit(5).show(truncate=False)

# 5. 查看时间范围
from pyspark.sql.functions import min, max, from_unixtime
time_range = raw_df.agg(
    min(from_unixtime("unixReviewTime")).alias("最早评论"),
    max(from_unixtime("unixReviewTime")).alias("最新评论")
)
time_range.show(truncate=False)

✅ 数据总行数: 1,689,188

🔍 核心字段空值检查:
   reviewText: 空值数 0 (0.00%)
   overall: 空值数 0 (0.00%)
   asin: 空值数 0 (0.00%)
   reviewerID: 空值数 0 (0.00%)

📊 评分(overall)分布检查:
+-------+-------+
|overall|  count|
+-------+-------+
|    1.0| 108725|
|    2.0|  82139|
|    3.0| 142257|
|    4.0| 347041|
|    5.0|1009026|
+-------+-------+


📈 helpful字段示例（[有用票数， 总票数]）:
+--------+
|helpful |
+--------+
|[0, 0]  |
|[12, 15]|
|[43, 45]|
|[9, 10] |
|[0, 0]  |
+--------+

+-------------------+-------------------+
|最早评论           |最新评论           |
+-------------------+-------------------+
|1999-06-13 00:00:00|2014-07-23 00:00:00|
+-------------------+-------------------+



In [8]:
# ============================================
# 任务1：数据清洗 (MapReduce/DataFrame实现)
# 输入：原始JSON数据 (raw_df)
# 输出：结构化的、洁净的DataFrame，并可保存为列式存储格式
# ============================================

from pyspark.sql.functions import col, from_unixtime, when

# 1. 数据清洗转换 - 注意这里采用一致的缩进
cleaned_df = (raw_df
    # 【可选】根据指南，过滤无效记录。由于reviewText无空值，此步可保留作为标准流程
    .filter(col("reviewText").isNotNull())
    # 解析helpful数组：提取有用票数和总票数
    .withColumn("helpful_votes", col("helpful")[0].cast("int"))
    .withColumn("total_votes", col("helpful")[1].cast("int"))
    # 计算“有用率”，避免除以零。后续计算用户影响力时会用到。
    .withColumn("helpful_ratio",
                when(col("total_votes") > 0, col("helpful_votes") / col("total_votes"))
                .otherwise(0.0)
               )
    # 转换Unix时间戳为标准的日期和时间格式，便于Hive按时间分区和查询
    .withColumn("review_date", from_unixtime("unixReviewTime", "yyyy-MM-dd"))
    .withColumn("review_year_month", from_unixtime("unixReviewTime", "yyyy-MM"))
    # 重命名列，使其更符合数据仓库的命名规范
    .withColumnRenamed("reviewerID", "user_id")
    .withColumnRenamed("asin", "product_id")
    .withColumnRenamed("overall", "rating")
    .withColumnRenamed("reviewText", "review_text")
    # 选择最终需要的列，构建一个清晰的事实表
    .select(
        "user_id",
        "product_id",
        "reviewerName",  # 暂时保留，后续可放入用户维度表
        "rating",
        "helpful_votes",
        "total_votes",
        "helpful_ratio",
        "review_text",
        "summary",
        "review_date",
        "review_year_month",
        "unixReviewTime"  # 保留原始时间戳，以防万一
    )
)

# 2. 查看清洗后的数据Schema和样本
print("=== 清洗后的数据Schema ===")
cleaned_df.printSchema()

print("\n=== 清洗后的数据样本（前5行）===")
cleaned_df.show(5, truncate=50)

# 3. 验证清洗结果
print(f"\n=== 清洗验证 ===")
print(f"清洗后数据行数: {cleaned_df.count():,} (应与原始数据一致: {raw_df.count():,})")
print("\n评分和有用票数统计摘要:")
cleaned_df.select("rating", "helpful_votes", "total_votes").describe().show()

=== 清洗后的数据Schema ===
root
 |-- user_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- helpful_ratio: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_year_month: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)


=== 清洗后的数据样本（前5行）===
+--------------+----------+------------------------+------+-------------+-----------+------------------+--------------------------------------------------+--------------------------------------+-----------+-----------------+--------------+
|       user_id|product_id|            reviewerName|rating|helpful_votes|total_votes|     helpful_ratio|                                       review_text|                               summary|r

In [9]:
# ===== 第一段：保存为Parquet格式（任务1的核心输出）=====
output_path = "./cleaned_reviews_parquet"
cleaned_df.write.mode("overwrite").parquet(output_path)
print(f"✅ 任务1完成！清洗后的数据已持久化到: {output_path}")

# 验证：从磁盘重新读取，证明数据已独立于内存存在
test_df = spark.read.parquet(output_path)
print(f"验证读取: {test_df.count()} 条记录，与原始一致")

✅ 任务1完成！清洗后的数据已持久化到: ./cleaned_reviews_parquet
验证读取: 1689188 条记录，与原始一致


In [10]:
# 将清洗后的DataFrame注册为Hive临时视图，命名为 fact_reviews
cleaned_df.createOrReplaceTempView("fact_reviews")
print("✅ 临时视图 'fact_reviews' 已创建，任务2可以开始！")

# 立即运行一个任务2的示例查询，验证衔接成功
print("\n🚀 示例：查询商品平均评分 (验证Hive SQL可用)")
spark.sql("""
    SELECT product_id, 
           AVG(rating) as avg_rating,
           COUNT(*) as review_count
    FROM fact_reviews
    GROUP BY product_id
    HAVING review_count >= 10
    ORDER BY avg_rating DESC
    LIMIT 5
""").show()

✅ 临时视图 'fact_reviews' 已创建，任务2可以开始！

🚀 示例：查询商品平均评分 (验证Hive SQL可用)
+----------+----------+------------+
|product_id|avg_rating|review_count|
+----------+----------+------------+
|B0000TFGWS|       5.0|          10|
|B00004TENT|       5.0|          14|
|B0000E68U2|       5.0|          12|
|B0000DYV9H|       5.0|          22|
|B000213ZFE|       5.0|          11|
+----------+----------+------------+

